### Use custom trained spaCy segmenter to segment CAP data

In [ ]:
# Load CAP data
# Filter out things formatting/section lines that only have roman numerals or single letters ("A", "II")
# Load segmenter
# Run each paragraph of each opinion through segmenter (alternatively, concat paragraphs if faster, but might be less accurate)
# If doing paragraphs only, account for introductions of quotes (e.g. "Section 123 says:\nxyz")